## Notebook Imports

In [1]:
import os, shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-09-14 09:21:31.913870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-14 09:21:31.913972: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Directories for Datasets

In [2]:
# location of original data
original_train_data = "/home/xviix/Desktop/Learning/dogs-vs-cats/train"
original_test_data = "/home/xviix/Desktop/Learning/dogs-vs-cats/test"


# directory to save new data
base_directory = "/home/xviix/Desktop/Learning/image-data"
try:
  os.mkdir(base_directory)
except FileExistsError as e:
  print(f"{e}: Base Directory Exists")

[Errno 17] File exists: '/home/xviix/Desktop/Learning/image-data': Base Directory Exists


In [3]:
train_dir = os.path.join(base_directory, "train")

test_dir = os.path.join(base_directory, "test")

val_dir = os.path.join(base_directory, "validation")

try:
  os.mkdir(train_dir)
  os.mkdir(test_dir)
  os.mkdir(val_dir)
except FileExistsError as e:
  print('Files Exists')



Files Exists


**Creating Training, Testing and Validation Sets for cats**

In [4]:
train_cats_dir = os.path.join(train_dir, "cats")
# test_cats_dir = os.path.join(test_dir, "cats")
val_cats_dir = os.path.join(val_dir, "cats")

**Creating Training, Testing and Validation Sets for dogs**

In [5]:
train_dogs_dir = os.path.join(train_dir, "dogs")
# test_dogs_dir = os.path.join(test_dir, "dogs")
val_dogs_dir = os.path.join(val_dir, "dogs")

try:
  os.mkdir(train_cats_dir)
  os.mkdir(val_cats_dir)
  os.mkdir(train_dogs_dir)
  os.mkdir(val_dogs_dir)
except Exception as e:
  print(e)

[Errno 17] File exists: '/home/xviix/Desktop/Learning/image-data/train/cats'


In [ ]:
file_names = [f"cat.{i}.jpg" for i in range(7000)]
for file_name in file_names:
  src = os.path.join(original_train_data, file_name)
  dest = os.path.join(train_cats_dir, file_name)
  shutil.copy(src, dest)

# validation set for cats
file_names = [f"cat.{i}.jpg" for i in range(7000, 12500)]
for file_name in file_names:
  src = os.path.join(original_train_data, file_name)
  dest = os.path.join(val_cats_dir, file_name)
  shutil.copy(src, dest)

In [ ]:
file_names = [f"dog.{i}.jpg" for i in range(7000)]
for file_name in file_names:
  src = os.path.join(original_train_data, file_name)
  dest = os.path.join(train_dogs_dir, file_name)
  shutil.copy(src, dest)

# validation set for dogs
file_names = [f"dog.{i}.jpg" for i in range(7000, 12500)]
for file_name in file_names:
  src = os.path.join(original_train_data, file_name)
  dest = os.path.join(val_dogs_dir, file_name)
  shutil.copy(src, dest)

In [8]:
# test set for cats
# file_names = [f"cat.{i}.jpg" for i in range(6250)]
for file_name in os.listdir(original_test_data):
  src = os.path.join(original_test_data, file_name)
  dest = os.path.join(test_dir, file_name)
  shutil.copy(src, dest)

## Model

**Model Instantiation**

In [6]:
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
  MaxPooling2D((2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Conv2D(128, (3, 3), activation="relu"),
  MaxPooling2D((2, 2)),
  Conv2D(128, (3, 3), activation="relu"),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(256, activation="relu"),
  Dense(1, activation="sigmoid")
])

2022-09-14 09:23:03.576190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-14 09:23:03.576226: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-14 09:23:03.576247: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xviix): /proc/driver/nvidia/version does not exist
2022-09-14 09:23:03.626275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [8]:
model.compile(
  loss="binary_crossentropy",
  optimizer="RMSProp",
  metrics=["accuracy"]
)

## Data Preprocessing

#### **1. Reading the Picture Files**

In [9]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [10]:
train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size=(150, 150),
  batch_size=20,
  class_mode="binary"
)

validation_generator = test_datagen.flow_from_directory(
  val_dir,
  target_size=(150,150),
  batch_size=20,
  class_mode="binary"
)

Found 14000 images belonging to 2 classes.
Found 11000 images belonging to 2 classes.


In [11]:
for data_batch, label_batch in train_generator:
  print(f"data batch shape:", data_batch.shape)
  print(f"label batch shape:", label_batch.shape)
  break

data batch shape: (20, 150, 150, 3)
label batch shape: (20,)


In [12]:
history = model.fit(
  train_generator,
  steps_per_epoch=150,
  epochs=15,
  validation_data=validation_generator,
  validation_steps=110
)

Epoch 1/15


2022-09-14 09:23:10.682646: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 56074240 exceeds 10% of free system memory.
2022-09-14 09:23:11.667714: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26542080 exceeds 10% of free system memory.
2022-09-14 09:23:12.197721: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19699200 exceeds 10% of free system memory.
2022-09-14 09:23:12.295565: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 23970816 exceeds 10% of free system memory.
2022-09-14 09:23:12.386547: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26542080 exceeds 10% of free system memory.


150/150 [==============================] - 149s 973ms/step - loss: 0.7171 - accuracy: 0.5287 - val_loss: 0.6808 - val_accuracy: 0.5323
Epoch 2/15
150/150 [==============================] - 139s 925ms/step - loss: 0.6754 - accuracy: 0.6050 - val_loss: 0.6165 - val_accuracy: 0.6677
Epoch 3/15
150/150 [==============================] - 186s 1s/step - loss: 0.6287 - accuracy: 0.6673 - val_loss: 0.5840 - val_accuracy: 0.6873
Epoch 4/15
150/150 [==============================] - 150s 1s/step - loss: 0.5794 - accuracy: 0.7020 - val_loss: 0.6457 - val_accuracy: 0.6859
Epoch 5/15
150/150 [==============================] - 142s 948ms/step - loss: 0.5709 - accuracy: 0.7087 - val_loss: 0.5026 - val_accuracy: 0.7536
Epoch 6/15
150/150 [==============================] - 139s 930ms/step - loss: 0.5398 - accuracy: 0.7267 - val_loss: 0.5413 - val_accuracy: 0.7273
Epoch 7/15
150/150 [==============================] - 139s 926ms/step - loss: 0.5120 - accuracy: 0.7530 - val_loss: 0.5317 - val_accuracy: 0.

In [ ]:
model.save("cats_and_dogs.h5")